In [ ]:
# @title Collegamento con Google Drive personale
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Import librerie
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
!pip install datasets
from datasets import Dataset
import pandas as pd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00


In [ ]:
# @title Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @title Preparazione tokenizer, modello e dataset
# Carica il tokenizer e il modello
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Aggiungi un token di padding al tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Carica il dataset CSV
df = pd.read_csv('/content/drive/MyDrive/Fake.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
test_size = int(len(df) * 0.1)
train_df = df[:-test_size]
test_df = df[-test_size:]

# Converti i DataFrame di pandas in un Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df[['text']])
test_dataset = Dataset.from_pandas(test_df[['text']])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
# @title Preparazione tokenizzazione ed addestramento
# Tokenizzazione
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=150)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Imposta i parametri di training
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    save_strategy="epoch",  # Salva il modello a ogni epoca
    save_total_limit=2,  # Mantieni solo gli ultimi 2 checkpoint
    eval_strategy="epoch",  # Usa eval_strategy invece di evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.001,
    report_to="none",
)

# Crea un Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,  # Dataset di test per la valutazione
)



Map:   0%|          | 0/21133 [00:00<?, ? examples/s]

Map:   0%|          | 0/2348 [00:00<?, ? examples/s]

In [ ]:
# @title Setup fine tuning quando non concluso
checkpoint_path = "/content/drive/MyDrive/results/checkpoint-84536"  # Sostituisci xxxx con il numero più alto

# Tokenizzazione
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=150)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Carica il modello pre-addestrato dal checkpoint
model = GPT2LMHeadModel.from_pretrained(checkpoint_path)

# Aggiorna i parametri per completare 10 epoche
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    save_strategy="epoch",
    save_total_limit=2,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=8,
    weight_decay=0.001,
    report_to="none",
)

# Riavvia il Trainer con il checkpoint caricato
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)



Map:   0%|          | 0/21133 [00:00<?, ? examples/s]

Map:   0%|          | 0/2348 [00:00<?, ? examples/s]

In [ ]:
# @title Avvia il fine-tuning (opzione ripresa fine tuning)
#trainer.train()

# Riprendi fine tuning
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Epoch,Training Loss,Validation Loss
4,2.714800,2.768471
5,2.635900,2.736880
6,2.487600,2.716520
7,2.459800,2.697680
8,2.376000,2.684472


TrainOutput(global_step=84536, training_loss=1.5815336577226933, metrics={'train_runtime': 7175.3601, 'train_samples_per_second': 23.562, 'train_steps_per_second': 11.781, 'total_flos': 1.29419168256e+16, 'train_loss': 1.5815336577226933, 'epoch': 8.0})

In [ ]:
# @title Valuta il modello sul test set
eval_results = trainer.evaluate()
print("Risultati della valutazione:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

# Funzione per generare fake news da descrizioni di test
def generate_fake_news(description):
    prompt = f"Create a fake news article based on this description: {description}"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(inputs, max_length=150, num_return_sequences=1,
                            do_sample=True, top_k=50, top_p=0.95, temperature=0.7)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Risultati della valutazione:
eval_loss: 2.6844587326049805
eval_model_preparation_time: 0.0023
eval_runtime: 1379.2929
eval_samples_per_second: 1.702
eval_steps_per_second: 0.213


In [ ]:
# Esempi di descrizioni per il testing
test_samples = [
    "a man holding a red umbrella in a park",
    "a woman cooking in a kitchen with a cat",
    "people gathered around a table at a restaurant",
    "there is a woman wearing a white hat and a black shirt",
]



In [ ]:
# Esegui il test di generazione
for description in test_samples:
    fake_news = generate_fake_news(description)
    print(f"Descrizione: {description}")
    print(f"Fake News Generata: {fake_news}\n")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Descrizione: a man holding a red umbrella in a park
Fake News Generata: Create a fake news article based on this description: a man holding a red umbrella in a park in Austin, Texas claims he saw an elderly woman walking down the street and saw a young woman in the middle of the street and say  Hey, hey    that s when he shot her.The woman was walking with her family, the young woman sitting next to her, and the man in the orange umbrella with the woman s name was driving.The man in the orange umbrella had a gun pointed at the woman and she got up and ran, according to the woman who is described as a dark-skinned woman with dark complexion. He then shot the woman in the leg, and she was pronounced dead in the ambulance.Austin Police say the



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Descrizione: a woman cooking in a kitchen with a cat
Fake News Generata: Create a fake news article based on this description: a woman cooking in a kitchen with a cat in the living room. The cat is in the kitchen and the mother is cooking. She has the cat in the living room.  A CNN reporter, Juan Gonzales, had the story published Wednesday afternoon and it was quickly debunked. The cat is not even there yet. The story was originally published on CNN s website on May 26, but it has since been updated to say that the mother is cooking the cat.The story was quickly debunked after CNN s Robert Parry learned that the woman who claimed the cat is in the kitchen is actually  a woman  and not a  cat.  CNN s Brian Stelter said the woman  



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Descrizione: people gathered around a table at a restaurant
Fake News Generata: Create a fake news article based on this description: people gathered around a table at a restaurant and wrote  fake news  under a black and white headline.  The article was published on Wednesday, August 26, 2017. It was authored by a white individual and posted on a white supremacist website. The article was published without any commentary from the author.The man, named as Ben Shapiro, posted the article on August 26. The story was based on the claim that President Trump made a  big mistake  by retweeting a story from CNN. The person who wrote the article, Ben Shapiro, was identified as a member of the National Policy Institute and was employed by Breitbart News.The National Policy Institute is a right-wing think tank that is funded

Descrizione: there is a woman wearing a white hat and a black shirt
Fake News Generata: Create a fake news article based on this description: there is a woman wearing a whit

In [ ]:
# Salva il modello fine-tuned
model.save_pretrained("/content/drive/MyDrive/fine_tuned_gpt2")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_gpt2")

('/content/drive/MyDrive/fine_tuned_gpt2/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_gpt2/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_gpt2/vocab.json',
 '/content/drive/MyDrive/fine_tuned_gpt2/merges.txt',
 '/content/drive/MyDrive/fine_tuned_gpt2/added_tokens.json')